## Evaluate effect of indexing on search rankings

In [1]:
import os
import pandas as pd

script_path = os.getcwd()
data_path = os.path.join(script_path,'data')
datafile = os.path.join('data','search rankings.tsv')
data = pd.read_csv(datafile,delimiter = '\t',header=0)
print(data.head(n=2))

                                          url in NDE last crawled  \
0  https://data.niaid.nih.gov/resources?id=ZENODO...    3/19/2024   
1  https://data.niaid.nih.gov/resources?id=OMICSD...    3/13/2024   

  indexed in google?  
0                 no  
1                 no  


In [ ]:
data['_id'] = data['url in NDE'].astype(str).str.replace("https://data.niaid.nih.gov/resources?id=","")
print(data.head(n=2))


In [ ]:
## get a random sample of the ids
sample_records = data.sample(250)
idlist = sample_records['_id'].unique().tolist()

In [ ]:
sample_records.to_csv(os.path.join('result','non-indexed_urls.tsv'),sep='\t',header=True)

In [ ]:
resultlist = []
fail = []
for eachid in idlist:
    try:
        r = requests.get(f"https://api.data.niaid.nih.gov/v1/query?q={eachid}&fields=_id,name")
        result = json.loads(r.text)
        hit = result['hits'][0]
        dataset_name = hit['name'].replace(" ","+").replace("<i>","").replace("</i>","")
        googleurl = f'https://www.google.com/search?q="{dataset_name}"'
        resultlist.append({'_id': hit['_id'],'name':hit['name'],'search_url':googleurl})
    except:
        fail.append(eachid)

resultdf = pd.DataFrame(resultlist)
print(resultdf.head(n=2))
print(len(resultdf))

In [ ]:
resultdf.to_csv(os.path.join('result','unindexed_datasets_to_evaluate.tsv'),sep='\t',header='True')

In [ ]:
more_variation = data.loc[~(data['_id'].str.contains('IMMPORT'))&~(data['_id'].str.contains('veupathdb'))]
print(len(more_variation))

In [ ]:
newidlist = more_variation['_id'].unique().tolist()

In [ ]:
new_resultlist = []
new_fail = []
for eachid in newidlist:
    try:
        r = requests.get(f"https://api.data.niaid.nih.gov/v1/query?q={eachid}&fields=_id,name")
        result = json.loads(r.text)
        hit = result['hits'][0]
        dataset_name = hit['name'].replace(" ","+").replace("<i>","").replace("</i>","")
        googleurl = f'https://www.google.com/search?q="{dataset_name}"'
        new_resultlist.append({'_id': hit['_id'],'name':hit['name'],'search_url':googleurl})
    except:
        new_fail.append(eachid)

new_resultdf = pd.DataFrame(new_resultlist)
print(new_resultdf.head(n=2))
print(len(new_resultdf))
new_resultdf.to_csv(os.path.join('result','more_unindexed_datasets_to_evaluate.tsv'),sep='\t',header='True')